In [535]:
from pathlib import Path
import configparser
from psycopg2 import connect
import pandas.io.sql as pandasql
from IPython.display import HTML
from psycopg2.extras import execute_values
import requests
from psycopg2.extras import RealDictCursor
import json
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [623]:
# Pull data from Database
count_sql = '''SELECT COUNT(*) 
                FROM gis_shared_streets.centreline_both_dir 
                WHERE FCODE_DESC IN ('Collector','Collector Ramp',
                'Local','Major Arterial','Major Arterial Ramp','Minor Arterial',
                'Minor Arterial Ramp','Pending') '''
with con:
    with con.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(count_sql)
            count_results = cur.fetchall()
            count = count_results[0]['count']
            
unmatched_gid = []
null_matched = []

for i in range(0, count, 500):
    get_centreline_sql = '''SELECT 'Feature' as type,  json_build_object('geo_id', geo_id, 'direction', direction) as properties, ST_AsGeoJson(dir_geom)::json as geometry 
    FROM gis_shared_streets.centreline_both_dir WHERE FCODE_DESC IN ('Collector','Collector Ramp',
    'Local','Major Arterial','Major Arterial Ramp','Minor Arterial',
    'Minor Arterial Ramp','Pending') ORDER BY geo_id LIMIT {} OFFSET {}'''.format(500, i)

    with con:
        with con.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(get_centreline_sql)
            results = cur.fetchall()
            final_json = json.dumps({"type": "FeatureCollection",
                                    "features": results})
            
# Send data to SharedStreets API    
    base_url = "https://api.sharedstreets.io/v0.1.0/match/geoms"
    apiKey = "bdd23fa1-7ac5-4158-b354-22ec946bb575"
    params = {'ignoreDirection': 'false',
              'bearingTolerance' : 45 ,
              'searchRadius' : 60,
              'auth' : apiKey,
              'tileHierarchy': 6,
              'dataSource': 'osm/planet-181029',
              'lengthTolerance' : 0.60,
              'snapTopology': 'true'
             }
    headers= {"Content-type": "application/json; charset=UTF-8"}
    
    r = requests.post(base_url, params = params, headers = headers, json = {"type": "FeatureCollection", "features": results})
    returned_json = r.json()
    feature = returned_json.get('matched').get('features')[0]
    unmatched = returned_json.get('unmatched').get('features')
    for feature in unmatched:
        unmatched_id = feature['properties']['geo_id']
        unmatched_direction = feature['properties']['direction']
        try:      
            unmatched_gid.append(str(int(unmatched_id)) + unmatched_direction)
        except Exception as e:
            print("unmatched: " + str(e))
            print(feature)
            print()
            print(unmatched_id)
    
# Send row to database
    features = returned_json.get('matched').get('features')
    rows = []
    for feature in features:
           
        if  feature['properties']['section'][0] > feature['properties']['section'][1]:
            feature['properties']['section'][0] = 0
            feature['properties']['section'][1] = 0
        if  feature['properties']['referenceId'] is None:
            null_matched.append(str(int(feature['properties']['geo_id']))+feature['properties']['direction'])
            continue
        
        
        
        row = (feature['properties']['referenceId'], 
                   feature['properties']['fromIntersectionId'],
                   feature['properties']['toIntersectionId'],
                   feature['properties']['roadClass'], 
                   feature['properties']['direction'],
                   feature['properties']['referenceLength'],
                   feature['properties']['side'], 
                   feature['properties'].get('score', None),
                   '[' + str(feature['properties']['section'][0]) +', '+ str(feature['properties']['section'][1]) +')', 
                   'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['coordinates'])+')', 
                   feature['properties']['originalFeature']['properties']['geo_id'],
                   feature['properties']['originalFeature']['properties']['direction'])
   

        rows.append(row)
        
        

    sql='INSERT INTO gis_shared_streets.centreline_both_dir_test (reference_id, from_intersection, to_intersection, road_class, direction, reference_length, side,score, section, geometry, geo_id, direction_centreline) VALUES %s'

    with con:
        with con.cursor() as cur:
            execute_values(cur, 
                           sql, 
                           rows) 
            
# Print unmatched features                
print(unmatched_gid)

['389Westbound', '389Eastbound', '465Eastbound', '475Eastbound', '475Westbound', '491Eastbound', '491Westbound', '932Eastbound', '932Westbound', '1237Southbound', '1341Northbound', '1342Westbound', '1349Southbound', '1349Northbound', '1533Westbound', '7381Westbound', '7381Eastbound', '7396Eastbound', '7508Eastbound', '7821Northbound', '8161Northbound', '8161Southbound', '8184Northbound', '8184Southbound', '8274Southbound', '8379Eastbound', '8379Westbound', '8537Southbound', '8682Northbound', '8682Southbound', '8741Southbound', '8821Southbound', '9406Westbound', '9406Eastbound', '9407Eastbound', '9407Westbound', '102865Southbound', '102865Northbound', '102868Westbound', '102868Eastbound', '102907Eastbound', '102907Westbound', '103192Eastbound', '103192Westbound', '103371Eastbound', '103371Westbound', '103422direction1', '103422direction2', '103558direction2', '103558direction1', '103638direction2', '103638direction1', '103774Southbound', '103774Northbound', '103898direction1', '103898di

In [577]:
len(unmatched_gid)

1689

In [617]:
# Pull data from Database
count = len(unmatched_gid)
           
unmatched_unmatched_gid = []
unmatched_matched_gid = []

for i in range(0, count, 500):
    get_centreline_sql = '''SELECT 'Feature' as type,  json_build_object('geo_id', geo_id, 'direction', direction) as properties, ST_AsGeoJson(dir_geom)::json as geometry 
    FROM gis_shared_streets.centreline_both_dir WHERE FCODE_DESC IN ('Collector','Collector Ramp',
    'Local','Major Arterial','Major Arterial Ramp','Minor Arterial',
    'Minor Arterial Ramp','Pending') AND geo_id||direction IN {} ORDER BY geo_id LIMIT {} OFFSET {}'''.format(tuple(unmatched_gid), 500, i)

    with con:
        with con.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(get_centreline_sql)
            results = cur.fetchall()
            final_json = json.dumps({"type": "FeatureCollection",
                                    "features": results})
            
# Send data to SharedStreets API    
    base_url = "https://api.sharedstreets.io/v0.1.0/match/geoms"
    apiKey = "bdd23fa1-7ac5-4158-b354-22ec946bb575"
    params = {'ignoreDirection': 'false',
              'bearingTolerance' : 45 ,
              'searchRadius' : 60,
              'auth' : apiKey,
              'tileHierarchy': 6,
              'dataSource': 'osm/planet-181029',
              'lengthTolerance' : 0.3,
              'snapTopology': 'true'
             }
    headers= {"Content-type": "application/json; charset=UTF-8"}
    
    r = requests.post(base_url, params = params, headers = headers, json = {"type": "FeatureCollection", "features": results})
    
    returned_json = r.json()
                      
   # feature = returned_json.get('matched').get('features')[0]
    unmatched = returned_json.get('unmatched').get('features')
    for feature in unmatched:
        unmatched_id = feature['properties']['geo_id']
        unmatched_direction = feature['properties']['direction']
        unmatched_unmatched_gid.append(str(int(unmatched_id)) + unmatched_direction)
        
        
    
# Send row to database
    features = returned_json.get('matched').get('features')
    if  feature['properties']['referenceId'] is None:
            null_matched.append(str(int(feature['properties']['geo_id']))+feature['properties']['direction'])
            continue
            
    for feature in features:
        matched_id = feature['properties']['originalFeature']['properties']['geo_id']
        matched_direction = feature['properties']['originalFeature']['properties']['direction']
        unmatched_matched_gid.append(str(int(matched_id)) + matched_direction)
    
    rows = []
    for feature in features:
           
        if  feature['properties']['section'][0] > feature['properties']['section'][1]:
            print("Error with geo_id {}".format(feature['properties']['originalFeature']['properties']['geo_id']))
            print(feature)
            feature['properties']['section'][0] = 0
            feature['properties']['section'][1] = 0
            
        row = (feature['properties']['referenceId'], 
                   feature['properties']['fromIntersectionId'],
                   feature['properties']['toIntersectionId'],
                   feature['properties']['roadClass'], 
                   feature['properties']['direction'],
                   feature['properties']['referenceLength'],
                   feature['properties']['side'], 
                   feature['properties'].get('score', None),
                   '[' + str(feature['properties']['section'][0]) +', '+ str(feature['properties']['section'][1]) +')', 
                   'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['coordinates'])+')', 
                   feature['properties']['originalFeature']['properties']['geo_id'],
                   feature['properties']['originalFeature']['properties']['direction'])

        rows.append(row)

        sql=' INSERT INTO gis_shared_streets.unmatched_centreline_both_dir (reference_id, from_intersection, to_intersection, road_class, direction, reference_length, side,score, section, geometry, geo_id, direction_centreline) VALUES %s'

   # with con:
        with con.cursor() as cur:
            execute_values(cur, 
                           sql, 
                           rows) 
            
# Print unmatched features                
print(unmatched_unmatched_gid)

['389Eastbound', '389Westbound', '465Eastbound', '475Westbound', '475Eastbound', '491Westbound', '491Eastbound', '932Westbound', '932Eastbound', '1237Southbound', '1341Northbound', '1342Westbound', '1349Northbound', '1349Southbound', '1533Westbound', '7381Eastbound', '7381Westbound', '7396Eastbound', '7508Eastbound', '7821Northbound', '8161Southbound', '8161Northbound', '8184Southbound', '8184Northbound', '8274Southbound', '8379Westbound', '8379Eastbound', '8537Southbound', '8682Northbound', '8682Southbound', '8741Southbound', '8821Southbound', '9406Westbound', '9406Eastbound', '9407Westbound', '9407Eastbound', '102865Northbound', '102865Southbound', '102868Westbound', '102868Eastbound', '102907Eastbound', '102907Westbound', '103192Eastbound', '103192Westbound', '103371Westbound', '103371Eastbound', '103422direction1', '103422direction2', '103558direction1', '103558direction2', '103638direction1', '103638direction2', '103774Southbound', '103774Northbound', '103898direction2', '103898di

In [612]:
len(unmatched_unmatched_gid)

1642

In [604]:
len(unmatched_gid)

1689

In [621]:
set(unmatched_matched_gid)

{'10010630Eastbound',
 '104709Eastbound',
 '105364Northbound',
 '105364Southbound',
 '112618Eastbound',
 '112618Westbound',
 '1141005Eastbound',
 '1141005Westbound',
 '1141112Northbound',
 '1141112Southbound',
 '1141210Northbound',
 '12335507Westbound',
 '14037319Eastbound',
 '14046456Eastbound',
 '14046456Westbound',
 '14172269Northbound',
 '14184507Eastbound',
 '14184507Westbound',
 '14234481Eastbound',
 '14234481Westbound',
 '20041963Northbound',
 '20042497Northbound',
 '30014021Eastbound',
 '30014021Westbound',
 '30064610Northbound',
 '30090510Westbound',
 '30104621Eastbound',
 '30104621Westbound',
 '3086139Southbound',
 '3369575Westbound',
 '3503403Westbound',
 '3633499Northbound',
 '3734666Westbound',
 '3735315Southbound',
 '3905080Eastbound',
 '3905080Westbound',
 '438775Northbound',
 '439745Southbound',
 '6765648Eastbound',
 '6765648Westbound',
 '7254144Northbound',
 '7254144Southbound',
 '7950452Eastbound',
 '7950452Westbound',
 '911733Eastbound',
 '911733Westbound',
 '913623W

In [ ]:
## getting kevin the link that got matched second time

In [584]:
count_sql = '''SELECT COUNT(*) 
                FROM gis.centreline 
                WHERE FCODE_DESC IN ('Collector','Collector Ramp',
                'Local','Major Arterial','Major Arterial Ramp','Minor Arterial',
                'Minor Arterial Ramp','Pending') '''
with con:
    with con.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(count_sql)
            count_results = cur.fetchall()
            count = count_results[0]['count']
            
unmatched_gid_kevin = []

for i in range(0, count, 500):
    get_centreline_sql = '''SELECT 'Feature' as type,  json_build_object('gid', gid) as properties, ST_AsGeoJson(geom)::json as geometry FROM gis.centreline WHERE FCODE_DESC IN ('Collector','Collector Ramp',
    'Local','Major Arterial','Major Arterial Ramp','Minor Arterial',
    'Minor Arterial Ramp','Pending') ORDER BY gid LIMIT {} OFFSET {}'''.format(500, i)

    with con:
        with con.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(get_centreline_sql)
            results = cur.fetchall()
            final_json = json.dumps({"type": "FeatureCollection",
                                    "features": results})
            
# Send data to SharedStreets API    
    base_url = "https://api.sharedstreets.io/v0.1.0/match/geoms"
    apiKey = "bdd23fa1-7ac5-4158-b354-22ec946bb575"
    params = {'ignoreDirection': 'false',
              'bearingTolerance' : 45 ,
              'searchRadius' : 60,
              'auth' : apiKey,
              'tileHierarchy': 6,
              'dataSource': 'osm/planet-181029',
              'lengthTolerance' : 0.60,
              'snapTopology': 'true'
             }
    headers= {"Content-type": "application/json; charset=UTF-8"}
    
    r = requests.post(base_url, params = params, headers = headers, json = {"type": "FeatureCollection", "features": results})
    returned_json = r.json()
    unmatched1 = returned_json.get('unmatched').get('features')
    for feature in unmatched1:
        unmatched_id1 = (feature['properties']['gid'])
        unmatched_gid_kevin.append(unmatched_id1)
    

            
# Print unmatched features                
print(unmatched_gid_kevin)

[64036, 64075, 64106, 64133, 64139, 64205, 64275, 64285, 64299, 64306, 64328, 64340, 64342, 64624, 64645, 64747, 64769, 64788, 64953, 65172, 65241, 65248, 65335, 65388, 65493, 65640, 65819, 65953, 65957, 65967, 66172, 66209, 66286, 66327, 66330, 66331, 66633, 66731, 66818, 66860, 66895, 67010, 67060, 67174, 67278, 67279, 67310, 67311, 67329, 67345, 67391, 67576, 67580, 67627, 67645, 67647, 67753, 67782, 67856, 67930, 67969, 68047, 68292, 68487, 68556, 68734, 68796, 68981, 69134, 69216, 69339, 69361, 69397, 69421, 69445, 69446, 69629, 69633, 69673, 69705, 69706, 69864, 69884, 70016, 70046, 70061, 70066, 70079, 70080, 70108, 70133, 70156, 70158, 70161, 70171, 70297, 70324, 70325, 70356, 70374, 70398, 70402, 70441, 70495, 70526, 70629, 70823, 70829, 70931, 70932, 70936, 70984, 71010, 71035, 71038, 71071, 71074, 71077, 71290, 71368, 71486, 71499, 71665, 71684, 71731, 71786, 72097, 72131, 72135, 72144, 72145, 72149, 72152, 72164, 72170, 72171, 72187, 72198, 72200, 72201, 72217, 72265, 72282

In [588]:
len(unmatched_gid_kevin)

2182

In [600]:
# Pull data from Database
count = len(unmatched_gid_kevin)
           
unmatched_unmatched_gid_kevin = []

for i in range(0, count, 500):
    get_centreline_sql = '''SELECT 'Feature' as type,  json_build_object('gid', gid) as properties, ST_AsGeoJson(geom)::json as geometry 
    FROM gis.centreline WHERE FCODE_DESC IN ('Collector','Collector Ramp',
    'Local','Major Arterial','Major Arterial Ramp','Minor Arterial',
    'Minor Arterial Ramp','Pending') AND gid IN {} ORDER BY gid LIMIT {} OFFSET {}'''.format(tuple(unmatched_gid_kevin), 500, i)

    with con:
        with con.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(get_centreline_sql)
            results = cur.fetchall()
            final_json = json.dumps({"type": "FeatureCollection",
                                    "features": results})
            
# Send data to SharedStreets API    
    base_url = "https://api.sharedstreets.io/v0.1.0/match/geoms"
    apiKey = "bdd23fa1-7ac5-4158-b354-22ec946bb575"
    params = {'ignoreDirection': 'false',
              'bearingTolerance' : 45 ,
              'searchRadius' : 60,
              'auth' : apiKey,
              'tileHierarchy': 6,
              'dataSource': 'osm/planet-181029',
              'lengthTolerance' : 0.60,
              'snapTopology': 'true'
             }
    headers= {"Content-type": "application/json; charset=UTF-8"}
    
    r = requests.post(base_url, params = params, headers = headers, json = {"type": "FeatureCollection", "features": results})
    
    returned_json = r.json()
    unmatched1 = returned_json.get('unmatched').get('features')
    for feature in unmatched1:
        unmatched_id1 = (feature['properties']['gid'])
        unmatched_unmatched_gid_kevin.append(unmatched_id1)                  
    

# Print unmatched features                
print(unmatched_unmatched_gid_kevin)

[64036, 64075, 64106, 64133, 64139, 64205, 64275, 64285, 64299, 64306, 64328, 64340, 64342, 64624, 64645, 64747, 64769, 64788, 64953, 65172, 65241, 65248, 65335, 65388, 65493, 65640, 65819, 65953, 65957, 65967, 66172, 66209, 66286, 66327, 66330, 66331, 66633, 66731, 66818, 66860, 66895, 67010, 67060, 67174, 67278, 67279, 67310, 67311, 67329, 67345, 67391, 67576, 67580, 67627, 67645, 67647, 67753, 67782, 67856, 67930, 67969, 68047, 68292, 68487, 68556, 68734, 68796, 68981, 69134, 69216, 69339, 69361, 69397, 69421, 69445, 69446, 69629, 69633, 69673, 69705, 69706, 69864, 69884, 70016, 70046, 70061, 70066, 70079, 70080, 70108, 70133, 70156, 70158, 70161, 70171, 70297, 70324, 70325, 70356, 70374, 70398, 70402, 70441, 70495, 70526, 70629, 70823, 70829, 70931, 70932, 70936, 70984, 71010, 71035, 71038, 71071, 71074, 71077, 71290, 71368, 71486, 71499, 71665, 71684, 71731, 71786, 72097, 72131, 72135, 72144, 72145, 72149, 72152, 72164, 72170, 72171, 72187, 72198, 72200, 72201, 72217, 72265, 72282

In [601]:
len(unmatched_unmatched_gid_kevin)

2182

In [661]:
# First time running
#null_matched= []
#unmatched_1 = matching(60, 0.60, 45, 1, None, 'gis_shared_streets.centreline_both_dir',  'gis_shared_streets.centreline_matched', con)
#print(unmatched_1)

# Second time running
#unmatched_2 = matching(60, 0.3, 45, 2, unmatched_1, 'gis_shared_streets.centreline_both_dir', 'gis_shared_streets.centreline_matched', con)
#len(unmatched_2)

# third time running
unmatched_3 = matching(60, 0.3, 25, 3, unmatched_2, 'gis_shared_streets.centreline_both_dir', 'gis_shared_streets.centreline_matched','gis_shared_streets.centreline_query_info', con)
len(unmatched_3)

0
[(3, 25, 60, 0.3, 1642, 0)]


1642

In [660]:

def matching(search_radius, length_tolerance, bearing_tolerance, query_num, previously_unmatched_id, input_table, output_table, query_info_table, primary_key1, primary_key2, con):
    
    # None -> if its first time
    # [] -> if everything has been matched 
    # [x,y, ... ]
    
    
    # eveyrthing was matched by previous calls to shared streets
    if previously_unmatched_id == []:
        return "Everything is matched"
    
    
    # get number of rows that need to be matched 
    
    if previously_unmatched_id is None: 
            
        count_sql = '''SELECT COUNT(*) 
                FROM gis_shared_streets.centreline_both_dir 
                WHERE FCODE_DESC IN ('Collector','Collector Ramp',
                'Local','Major Arterial','Major Arterial Ramp','Minor Arterial',
                'Minor Arterial Ramp','Pending') '''
            
        with con:
            with con.cursor(cursor_factory=RealDictCursor) as cur:
                    cur.execute(count_sql)
                    count_results = cur.fetchall()
                    count = count_results[0]['count']
                    
                    
    else: 
        count = len(previously_unmatched_id)
            
    
    
    # list for all the ids that do not get matched 
    unmatched_rows = []
    
    for i in range(0, count, 500):
        
        #print("i right now" + str(i))
        
        if previously_unmatched_id is None: 
            # first time calling function on input table 
            get_centreline_sql = '''SELECT 'Feature' as type,  json_build_object('geo_id', geo_id, 'direction', direction) as properties, ST_AsGeoJson(dir_geom)::json as geometry 
            FROM ''' + input_table + '''
            WHERE FCODE_DESC IN ('Collector','Collector Ramp',
            'Local','Major Arterial','Major Arterial Ramp','Minor Arterial',
            'Minor Arterial Ramp','Pending') ORDER BY geo_id LIMIT 500 OFFSET {}'''.format(i)
            
        else:
            # you have already called the function on input table (or tried matching table using shared streets somehow)
            get_centreline_sql = '''SELECT 'Feature' as type,  json_build_object('geo_id', geo_id, 'direction', direction) as properties, ST_AsGeoJson(dir_geom)::json as geometry 
            FROM gis_shared_streets.centreline_both_dir WHERE FCODE_DESC IN ('Collector','Collector Ramp',
            'Local','Major Arterial','Major Arterial Ramp','Minor Arterial',
            'Minor Arterial Ramp','Pending') AND geo_id||direction IN {} ORDER BY geo_id LIMIT 500 OFFSET {}'''.format(tuple(previously_unmatched_id), i)

        with con:
            with con.cursor(cursor_factory=RealDictCursor) as cur:
                cur.execute(get_centreline_sql)
                results = cur.fetchall()
                final_json = json.dumps({"type": "FeatureCollection",
                                        "features": results})

    # Send data to SharedStreets API    
        base_url = "https://api.sharedstreets.io/v0.1.0/match/geoms"
        apiKey = "bdd23fa1-7ac5-4158-b354-22ec946bb575"
        params = {'ignoreDirection': 'false',
                  'bearingTolerance' : bearing_tolerance,
                  'searchRadius' : search_radius,
                  'auth' : apiKey,
                  'tileHierarchy': 6,
                  'dataSource': 'osm/planet-181029',
                  'lengthTolerance' : length_tolerance,
                  'snapTopology': 'true'
                 }
        headers= {"Content-type": "application/json; charset=UTF-8"}

        r = requests.post(base_url, params = params, headers = headers, json = {"type": "FeatureCollection", "features": results})
        returned_json = r.json()
        unmatched = returned_json.get('unmatched').get('features')
        for feature in unmatched:
            unmatched_id = feature['properties']['geo_id']
            unmatched_direction = feature['properties']['direction']
            unmatched_rows.append(str(int(unmatched_id)) + unmatched_direction)


    # Send row to database
        features = returned_json.get('matched').get('features')
        rows = []
        for feature in features:

            if  feature['properties']['section'][0] > feature['properties']['section'][1]:
                feature['properties']['section'][0] = 0
                feature['properties']['section'][1] = 0
                
            if  feature['properties']['referenceId'] is None:
                null_matched.append(str(int(feature['properties']['geo_id']))+feature['properties']['direction'])
                continue



            row = (feature['properties']['referenceId'], 
                       feature['properties']['fromIntersectionId'],
                       feature['properties']['toIntersectionId'],
                       feature['properties']['roadClass'], 
                       feature['properties']['direction'],
                       feature['properties']['referenceLength'],
                       feature['properties']['side'], 
                       feature['properties'].get('score', None),
                       '[' + str(feature['properties']['section'][0]) +', '+ str(feature['properties']['section'][1]) +')', 
                       'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['coordinates'])+')', 
                       feature['properties']['originalFeature']['properties']['geo_id'],
                       feature['properties']['originalFeature']['properties']['direction'], 
                       query_num)


            rows.append(row)



        sql='INSERT INTO ' + output_table + '(reference_id, from_intersection, to_intersection, road_class, direction, reference_length, side,score, section, geometry, geo_id, direction_centreline, query_num) VALUES %s'
    

        with con:
            with con.cursor() as cur:
                execute_values(cur, 
                               sql, 
                               rows) 
    
   

    print(count-len(unmatched_rows))
    
    query_info = [(query_num,bearing_tolerance, search_radius, length_tolerance, count, count-len(unmatched_rows))]
    
    print(query_info)
    
    sql ='INSERT INTO ' + query_info_table +'(query_num, bearing_tolerance, search_radius, length_tolerance, total_rows, matched_rows) VALUES %s'
    with con:
            with con.cursor() as cur:
                execute_values(cur, 
                               sql, 
                               query_info) 
    
    return unmatched_rows

In [662]:
def create_query_info_table(layer_name):
    sql = 'CREATE TABLE gis_shared_streets.'+layer_name+'_query_info_table (like gis_shared_streets.platonic_ideal_query_info_table)'